In [7]:
import os
import pandas as pd
import librosa
from eda import Mixes, Tracks

In [8]:
def get_audio_features(audio_path):
    track_id = os.path.splitext(os.path.basename(audio_path))[0]
    track_ext = os.path.splitext(os.path.basename(audio_path))[1]

    audio_data, sr = librosa.load(audio_path, sr=None)
    duration = librosa.get_duration(y=audio_data, sr=sr)
    num_channels = audio_data.shape[1] if len(audio_data.shape) > 1 else 1
    amplitude_range = (audio_data.min(), audio_data.max())
    spectral_centroid = librosa.feature.spectral_centroid(y=audio_data, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio_data)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sr)
    features = {
        'track_id': track_id,
        'sample_rate': sr,
        'duration': duration,
        'num_channels': num_channels,
        'amplitude_range': amplitude_range,
        'spectral_centroid': spectral_centroid,
        'zero_crossing_rate': zero_crossing_rate,
        'mel_spectrogram': mel_spectrogram,
        'track_ext': track_ext,
        'input_path': audio_path
    }
    return features


In [9]:
# folder_path = '../audio/'
# track_features = {}

# for file_name in os.listdir(folder_path):
#     if file_name.endswith('.wav') or file_name.endswith('.mp3') or file_name.endswith('.flac') or file_name.endswith('.aif'):
#         file_path = os.path.join(folder_path, file_name)

#         features = get_audio_features(file_path)

#         track_id = os.path.splitext(file_name)[0]
#         track_features[track_id] = features

In [10]:
# track_features

In [11]:
# train_folder_path = '../data/tracks/'
# output_folder_path = '../data/'
# batch_features = []

# for i, file_name in enumerate(os.listdir(train_folder_path)):

#     if file_name.endswith('.mp3'):

#         file_path = os.path.join(train_folder_path, file_name)

#         features = get_audio_features(file_path)

#         track_id = os.path.splitext(file_name)[0]
#         batch_features.append({'track_id': track_id, **features})

#         # Save features in CSV file every batch_size files or on the last iteration
#         if (i + 1) % 300 == 0 or i == len(os.listdir(train_folder_path)) - 1:
#             df = pd.DataFrame(batch_features)

#             batch_number = int((i + 1) / 300)  # Calculate batch number
#             csv_filename = f'batch_{batch_number}.csv'
#             csv_filepath = os.path.join(output_folder_path, csv_filename)
#             df.to_csv(csv_filepath, index=False)

#             # Clear the batch features list
#             batch_features = []


In [12]:
train_folder_path = '../data/tracks/'
output_folder_path = '../data/'

mixes = Mixes('../data/djmix-dataset.json', 5, 20)
tracks = Tracks(mixes)

mix_features = {}
batch_size = 300
batch_count = 0


for file_name in os.listdir(train_folder_path):

    if file_name.endswith('.mp3'):

        file_path = os.path.join(train_folder_path, file_name)
        track_id = os.path.splitext(file_name)[0]
        
        mix_id = mixes.get(track_id, 'test')

        features = get_audio_features(file_path)
        
        if mix_id not in mix_features:
            mix_features[mix_id] = {}
        
        mix_features[mix_id][track_id] = {
            'track_id': track_id,
            'mix_id': mix_id,
            'features': features
        }
        if len(mix_features[mix_id]) % batch_size == 0:
            df = pd.DataFrame(mix_features[mix_id]).transpose()
            batch_count += 1
            csv_filename = f'{mix_id}_{batch_count}.csv'
            csv_filepath = os.path.join(output_folder_path, csv_filename)
            df.to_csv(csv_filepath)

# Export track features to separate CSV files for each mix
for mix_id, track_features in mix_features.items():

    df = pd.DataFrame(track_features).transpose()

    csv_filename = f'{mix_id}.csv'
    csv_filepath = os.path.join(output_folder_path, csv_filename)
    df.to_csv(csv_filepath)


/var/folders/sn/zz32wyd938j2bblqhcs5jn180000gn/T/ipykernel_25630/455363276.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(audio_path, sr=None)
[src/libmpg123/parse.c:skip_junk():1276] error: Giving up searching valid MPEG header after 65536 bytes of junk.
/Users/bjornbraun/miniconda3-m1/miniconda-m1/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/var/folders/sn/zz32wyd938j2bblqhcs5jn180000gn/T/ipykernel_25630/455363276.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(audio_path, sr=None)
[src/libmpg123/parse.c:skip_junk():1276] error: Giving up searching valid MPEG header after 65536 bytes of junk.
/Users/bjornbraun/miniconda3-m1/miniconda-m1/lib/python3.10/site-packages/librosa/cor

: 

: 